In [31]:
### SHERIFF TAIWO 
### SYS 6014 -- Decision Analysis 

import warnings
import numpy as np
warnings.simplefilter(action='ignore', category=FutureWarning)
print('x' in np.arange(5))   #returns False, without Warning
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import os 
from IPython.display import display, HTML

#import geopy.distance

file_encoding = 'cp1252'  # set file_encoding to the file encoding (utf8, latin1, etc.)

#  Read in raw data from store
#raw_df = pd.read_excel("raw_df.xlsx", sheet_name = 'Raw_Data')


# outstanding mystery are people with null MATRICULANT and MATRICULANTS = 1
# What is SCHOOL_NBR, DISCOUNT_RATE?
# Consider International TOEFL Missing Flag
# COUNTRY, STATE, COUNTY, ZIP_CODE have ~ 50,000 missing
# LEVL has only 274 NaN rest are H should probably drop it
# DEP_STATUS is missing if FAFSA is missing


os.getcwd()
##os.chdir('/home/jovyan/project')
##display(HTML(df2.to_html()))

False


'/home/jovyan/project'

In [32]:

# load raw data
raw_df = pd.read_excel(
    'All_Students.xlsx',
    sheet_name = "all_students",
    encoding=file_encoding,
    parse_dates=['APPLICATION_DATE',
                 'ADMITTANCE_DATE',
                 'DEPOSIT_DATE',
                 'APPLICATION_REVIEW_DATE',
                 'MATRICULANT_DT',
                 'EDW_CREATE_DATE'])

raw_df.loc[raw_df.ENTY_ID == -1, 'ENTY_ID'] = None  # convert -1 ENTY_IDs into missing value
raw_df.loc[raw_df.SCHOOL_NBR == -1, 'SCHOOL_NBR'] = None  # convert -1 SCHOOL_NBR into missing value
cols_to_str = [  # columns that contain numeric identifiers to convert to 0 padded strings
    ('PID', 7),
    ('AIDYR', 4),
    ('RECRUIT_ZIP_CD', 5),
    ('ENTY_ID', 6),
    ('COLGE_CD', 2),
    ('SCHOOL_NBR', 6)
]
for col, str_len in cols_to_str:
    raw_df[col] = raw_df[col].map(lambda x: str(x).rjust(str_len, '0'))

cols_to_drop = ['ACTUAL_TARGET',  # non-varying
                'RECRUIT_COUNTY_STATE',  # redundant
                'PARENT_EDUCATIONAL_ATTAINMENT',  # missing
                'APPLICATION_DT_CURRENT_YR',  # missing
                'ADMITTANCE_DT_CURRENT_YR',  # missing
                'DEPOSIT_DATE_CURRENT_YEAR',  # missing
                'PID_1',  # redundant
                'ACADEMIC_QUALITY',  # missing
                'MELT_FLAG',  # missing
                'PROSPECT_FLAG',  # missing
                'PROSPECT_DATE',  # missing
                'MERIT_AID_TOTAL',  # missing
                'NEED_AID_TOTAL',  # missing
                'MERIT_TYPE_1',  # missing
                'MERIT_TYPE_2',  # missing
                'MERIT_TYPE_3',  # missing
                'MERIT_TYPE_4',  # missing
                'MERIT_TYPE_5',  # missing
                'APPLICANT',  # non-varying
                'APPLICATIONS',  # non-varying
                'ADMITTANCE_DATE_CURRENT_YEAR',  # missing
                'DEPOSITS',  # redundant
                'COLGE_CD_1',  # redundant
                'TGT_APPLICATIONS',  # missing
                'TGT_ADMITS',  # missing
                'TGT_DEPOSITS',  # missing
                'TGT_MATRICULANTS',  # missing
                'TGT_TERM_CD',  # missing
                'TGT_DT',  # missing
                'IN_STATE_TUITION',  # missing
                'APPLICATION_TERM_1',  # redundant
                'ADMIT_TERM',  # redundant to APPLICATION_TERM
                'FIRST_SEMESTER_GPA',  # missing
                'FED_PERKINS_LOAN',  # missing
                'PSAT'  # missing
                ]
implct_zero = [  # These are columns with large numbers of missing values and no explicit 0s
    'AP_CNT_NBR',
    'TUITION_WAIVER',
    'ATHLETIC_AID',
    'DISCOUNT_RATE',
    'PELL_GRANT',
    'FED_GIFT_EXLUDING_PELL_AID',
    'FED_SUB_LOAN',
    'FED_UNSUB_LOAN',
    'PLUS_LOAN',
    'STATE_GIFT_AID',
    'MSU_OFR_GIFT_AID',
    'OTHER_GIFT',
    'OTHER_LOAN',
    'MSU_ACPT_GIFT_AID'
    ]
implct_no = [  # These are columns with large numbers of missing values and no explicit 'N's
    'FAFSA',
    'FINANCIAL_AID_FLAG',
    'FIRST_GENERATION',
    'LEGACY_STUDENT',
    'MATRICULANT'
]
raw_df[implct_zero] = raw_df[implct_zero].fillna(0)
raw_df[implct_no] = raw_df[implct_no].fillna('N')
raw_df = raw_df.drop(cols_to_drop, axis=1)  # remove extraneous columns without data
raw_df = raw_df.sort_values(  # remove duplicate values keeping the most recent application_date per PID
    'APPLICATION_DATE', ascending=False)
raw_df = raw_df.drop_duplicates(subset='PID')
list(raw_df.columns.values) ### Print header names 

['PID',
 'LEVL',
 'APPLICATION_TERM',
 'CITIZENSHIP',
 'ETHNICITY',
 'GENDER',
 'COUNTRY',
 'STATE',
 'COUNTY',
 'ZIP_CODE',
 'SAT_TYPE',
 'INTENDED_DECLARED_MAJOR',
 'APPLICATION_DATE',
 'ADMITTANCE_DATE',
 'DEPOSIT_DATE',
 'MATRICULANT',
 'DEP_STATUS',
 'APPLICATION_REVIEW_STATUS',
 'APPLICATION_REVIEW_DATE',
 'SCHOOL_NAME',
 'SCHOOL_CEEB',
 'SCHOOL_TYPE',
 'SCHOOL_ZIP_CODE',
 'SCHOOL_STATE',
 'SCHOOL_CITY',
 'SCHOOL_COUNTRY',
 'COLLEGE',
 'RECRUIT_CD',
 'RECRUITER_MSU_ID',
 'TERRITORY_NM',
 'TARGET_AREA',
 'RECRUIT_FRST_NM',
 'RECRUIT_LST_NM',
 'NEED_LEVEL_EFC_BAND_CATEGORY',
 'MATRICULANT_DT',
 'EDW_CREATE_DATE',
 'STUDENT_SEGMENT',
 'RECRUIT_COUNTY',
 'RECRUIT_COUNTRY',
 'RECRUIT_CITY',
 'RECRUIT_STATE',
 'RECRUIT_COUNTRY_NAME',
 'FEE_BSD_RES_TYP_CD',
 'NEED_LEVEL_DEFINITION',
 'HIGH_SCHOOL_GPA',
 'ACT_COMPOSITE',
 'ACT_ENGLISH',
 'ACT_MATH',
 'ACT_READING',
 'ACT_SCIENCE',
 'ACT_WRITING',
 'SAT_MATH',
 'SAT_VERBAL',
 'SAT_WRITING',
 'IELTS_SCORE',
 'FIRST_GENERATION',
 'LEGACY_ST

In [13]:
display(raw_df)

,PID,LEVL,APPLICATION_TERM,CITIZENSHIP,ETHNICITY,GENDER,COUNTRY,STATE,COUNTY,ZIP_CODE,SAT_TYPE,INTENDED_DECLARED_MAJOR,APPLICATION_DATE,ADMITTANCE_DATE,DEPOSIT_DATE,MATRICULANT,DEP_STATUS,APPLICATION_REVIEW_STATUS,APPLICATION_REVIEW_DATE,SCHOOL_NAME,SCHOOL_CEEB,SCHOOL_TYPE,SCHOOL_ZIP_CODE,SCHOOL_STATE,SCHOOL_CITY,SCHOOL_COUNTRY,COLLEGE,RECRUIT_CD,RECRUITER_MSU_ID,TERRITORY_NM,TARGET_AREA,RECRUIT_FRST_NM,RECRUIT_LST_NM,NEED_LEVEL_EFC_BAND_CATEGORY,MATRICULANT_DT,EDW_CREATE_DATE,STUDENT_SEGMENT,RECRUIT_COUNTY,RECRUIT_COUNTRY,RECRUIT_CITY,RECRUIT_STATE,RECRUIT_COUNTRY_NAME,FEE_BSD_RES_TYP_CD,NEED_LEVEL_DEFINITION,HIGH_SCHOOL_GPA,ACT_COMPOSITE,ACT_ENGLISH,ACT_MATH,ACT_READING,ACT_SCIENCE,ACT_WRITING,SAT_MATH,SAT_VERBAL,SAT_WRITING,IELTS_SCORE,FIRST_GENERATION,LEGACY_STUDENT,ADMITTANCE,DEPOSIT,AIDYR,FAFSA,EFC1,TOTAL_AGI1,STUDENT_AGI1,PARENT_AGI1,PELL_GRANT,FED_GIFT_EXLUDING_PELL_AID,FED_SUB_LOAN,FED_UNSUB_LOAN,FED_TOTAL_LOAN,PLUS_LOAN,STATE_GIFT_AID,MSU_OFR_GIFT_AID,OTHER_GIFT,OTHER_LOAN,TUITION,ROOM,TRANSPORTATION,BOOKS,OTHER,SAT_COMBINED,DISCOUNT_RATE,AP_CNT_NBR,FINANCIAL_AID_TOTAL,FINANCIAL_AID_FLAG,COA,NTR_BASED_ON_OFFERED,NTR_ACCEPTED,PLACED_INTO_CALCULUS,PLACED_INTO_COLLEGE_WRITING,PLACED_INTO_COLLEGE_MATH,ADMITS,MATRICULANTS,ENTY_ID,COLGE_CD,SCHOOL_NBR,RECRUIT_ZIP_CD,LATD_NBR,LGTD_NBR,TOEFL_SCORE,ATHLETIC_AID,TUITION_WAIVER,PRED_GPA_NBR,MATH_PLACEMENT_SCOR_NBR,MSU_ACPT_GIFT_AID,Days_To_Admit,Days_Since_Application_Review,Days_To_Deposit,Yield_Indicator
28555,2750786,H,FS19,U.S. Citizen,Asian,F,NaN,NaN,NaN,NaN,SATX,Political Science,2019-05-17,2019-05-20,NaT,N,NaN,CMPL,2019-05-20,Glastonbury High School,70215,H,06033,CT,GLASTONBURY,US,Social Science,R10,larrya,New York Metro/DC Metro,OTHER/NON TARGET,Larry,Alterman,HIGH,NaT,2019-07-02,OUT OF STATE,Hartford County,US,GLASTONBURY,CT,United States,O,EFC < 20% of COA,4.303,0,0,0,0,0,0,770,680,0,0.0,0,0,1,0,2020,0,0,0,0,0,0,0.00,0,0,0,0,0,10000.0,0.0,0.0,40446,10502,1250,1134,2576,1450,24.762282,0,10000.00,1,55908,45908.0,45908.0,0,0,0,1,0,30998.0,38,30998.0,06033,41.70,-72.60,0.0,0.0,0,3.442,0,10000.0,3,3,0,0
56920,2743440,H,FS19,U.S. Citizen,White,M,US,MI,WAYN,48212,NaN,Electrical Engineering,2019-05-15,2019-06-25,2019-06-23,E,NaN,CMPL,2019-06-25,Frontier International Academy,231841,H,48212,MI,HAMTRAMCK,US,Engineering,313,scarber,ZONE 10B,NaN,Brandon,Scarber,HIGH,2019-06-28,2019-07-02,IN STATE,Wayne,US,Hamtramck,MI,United States,I,EFC < PELL CUTOFF,2.710,0,0,0,0,0,0,0,0,0,0.0,1,0,1,1,2020,0,400,44013,0,44013,5845,0.00,3500,2000,5500,0,0,8300.0,0.0,0.0,15708,10502,464,1134,2576,0,53.048703,0,19645.00,1,30384,22084.0,22084.0,0,0,0,1,1,100637.0,16,100637.0,48212,42.38,-83.10,0.0,0.0,0,2.168,0,8300.0,41,41,39,1
18798,2589512,H,FS19,U.S. Citizen,White,M,NaN,NaN,NaN,NaN,SATX,Chemical Engineering,2019-05-13,2019-06-18,NaT,N,D,CMPL,2019-06-18,Indian Hill High School,361020,H,45243,OH,CINCINNATI,US,Engineering,R2,irissvb,Ohio/Indiana/Kentucky,Cincinnati Area,Iris,Shen-Van Buren,MODERATE,NaT,2019-07-02,OUT OF STATE,Hamilton County,US,CINCINNATI,OH,United States,O,EFC >= 20% of COA and < 60%,4.488,0,0,0,0,0,0,700,660,0,0.0,0,1,1,0,2020,1,24311,116651,1585,115066,0,0.00,3500,2000,5500,0,0,18750.0,0.0,0.0,40446,10502,1250,1134,2576,1360,46.429279,0,24250.00,1,55908,37158.0,37158.0,0,0,0,1,0,54729.0,16,54729.0,45243,39.09,-84.51,0.0,0.0,0,3.590,0,18750.0,36,36,0,0
49426,2612448,H,FS19,U.S. Citizen,Black or African American,F,NaN,NaN,NaN,NaN,SATX,Hospitality Business,2019-05-07,2019-05-20,NaT,N,D,CMPL,2019-05-20,Communication & Media Arts High School,230874,H,48227,MI,DETROIT,US,Business,313,scarber,ZONE 10B,NaN,Brandon,Scarber,HIGH,NaT,2019-07-02,IN STATE,Wayne,US,Detroit,MI,United States,I,EFC < PELL CUTOFF,3.097,0,0,0,0,0,0,510,540,0,0.0,1,0,1,0,2020,1,0,25910,0,25910,6195,550.00,3500,2000,5500,0,0,10800.0,0.0,0.0,15708,10502,464,1134,2576,1050,69.027227,0,23045.00,1,30384,19584.0,19584.0,0,0,0,1,0,43988.0,08,43988.0,48227,42.21,-83.15,0.0,0.0,0,2.319,0,10800.0,13,13,0,0
8841,2739797,H,FS1

In [21]:
# create days since may 1 synthetic feature
raw_df['days_since_may'] = abs(raw_df.APPLICATION_TERM.map(lambda x:
                                                           pd.datetime.strptime(
                                                               '05-01-' + x.replace('FS', ''),
                                                                '%m-%d-%y')) - raw_df.APPLICATION_DATE).map(lambda x: x.days)
raw_df['need'] = raw_df.COA - raw_df.EFC1
raw_df.loc[raw_df.need <= 0, 'need'] = 0
raw_df['FINANCIAL_AID_TOTAL'] = raw_df.FINANCIAL_AID_TOTAL - raw_df.MSU_ACPT_GIFT_AID  #FINANCIAL_AID Total excluding Igrant 
raw_df['Yield_Indicator'] = raw_df.MATRICULANTS

# Create Best Score Synthetic Feature
sat_bins = [0, 500, 550, 590, 620, 650, 690, 730, 780, 830, 880, 920, 960, 990, 1030, 1060, 1100, 1130, 1160, 1200,
            1230, 1260, 1300, 1330, 1360, 1390, 1420, 1450, 1490, 1530, 1570, 1601]
raw_df['SAT_CONV'] = pd.cut(raw_df.SAT_COMBINED, bins= sat_bins, right=False, labels=False) + 6
raw_df.loc[raw_df.SAT_CONV == 6, 'SAT_CONV'] = 5  # correct because lowest category is 5 not 6
raw_df['BEST_SCORE'] = raw_df[['SAT_CONV', 'ACT_COMPOSITE']].max(axis=1, skipna=True)  # take highest score no NA

# Create School Feeder Synthetic Feature
school_count = raw_df.groupby(  # create dataframe with school identifier and number of applicants
    'SCHOOL_CEEB').size().reset_index().rename(columns={0: 'school_feeder'})

In [33]:
raw_df = raw_df.merge(school_count, on='SCHOOL_CEEB', how = 'left')

In [34]:
display(raw_df)
display(school_count)

,PID,LEVL,APPLICATION_TERM,CITIZENSHIP,ETHNICITY,GENDER,COUNTRY,STATE,COUNTY,ZIP_CODE,SAT_TYPE,INTENDED_DECLARED_MAJOR,APPLICATION_DATE,ADMITTANCE_DATE,DEPOSIT_DATE,MATRICULANT,DEP_STATUS,APPLICATION_REVIEW_STATUS,APPLICATION_REVIEW_DATE,SCHOOL_NAME,SCHOOL_CEEB,SCHOOL_TYPE,SCHOOL_ZIP_CODE,SCHOOL_STATE,SCHOOL_CITY,SCHOOL_COUNTRY,COLLEGE,RECRUIT_CD,RECRUITER_MSU_ID,TERRITORY_NM,TARGET_AREA,RECRUIT_FRST_NM,RECRUIT_LST_NM,NEED_LEVEL_EFC_BAND_CATEGORY,MATRICULANT_DT,EDW_CREATE_DATE,STUDENT_SEGMENT,RECRUIT_COUNTY,RECRUIT_COUNTRY,RECRUIT_CITY,RECRUIT_STATE,RECRUIT_COUNTRY_NAME,FEE_BSD_RES_TYP_CD,NEED_LEVEL_DEFINITION,HIGH_SCHOOL_GPA,ACT_COMPOSITE,ACT_ENGLISH,ACT_MATH,ACT_READING,ACT_SCIENCE,ACT_WRITING,SAT_MATH,SAT_VERBAL,SAT_WRITING,IELTS_SCORE,FIRST_GENERATION,LEGACY_STUDENT,ADMITTANCE,DEPOSIT,AIDYR,FAFSA,EFC1,TOTAL_AGI1,STUDENT_AGI1,PARENT_AGI1,PELL_GRANT,FED_GIFT_EXLUDING_PELL_AID,FED_SUB_LOAN,FED_UNSUB_LOAN,FED_TOTAL_LOAN,PLUS_LOAN,STATE_GIFT_AID,MSU_OFR_GIFT_AID,OTHER_GIFT,OTHER_LOAN,TUITION,ROOM,TRANSPORTATION,BOOKS,OTHER,SAT_COMBINED,DISCOUNT_RATE,AP_CNT_NBR,FINANCIAL_AID_TOTAL,FINANCIAL_AID_FLAG,COA,NTR_BASED_ON_OFFERED,NTR_ACCEPTED,PLACED_INTO_CALCULUS,PLACED_INTO_COLLEGE_WRITING,PLACED_INTO_COLLEGE_MATH,ADMITS,MATRICULANTS,ENTY_ID,COLGE_CD,SCHOOL_NBR,RECRUIT_ZIP_CD,LATD_NBR,LGTD_NBR,TOEFL_SCORE,ATHLETIC_AID,TUITION_WAIVER,PRED_GPA_NBR,MATH_PLACEMENT_SCOR_NBR,MSU_ACPT_GIFT_AID,Days_To_Admit,Days_Since_Application_Review,Days_To_Deposit,Yield_Indicator,school_feeder
0,2750786,H,FS19,U.S. Citizen,Asian,F,NaN,NaN,NaN,NaN,SATX,Political Science,2019-05-17,2019-05-20,NaT,N,NaN,CMPL,2019-05-20,Glastonbury High School,70215,H,06033,CT,GLASTONBURY,US,Social Science,R10,larrya,New York Metro/DC Metro,OTHER/NON TARGET,Larry,Alterman,HIGH,NaT,2019-07-02,OUT OF STATE,Hartford County,US,GLASTONBURY,CT,United States,O,EFC < 20% of COA,4.303,0,0,0,0,0,0,770,680,0,0.0,0,0,1,0,2020,0,0,0,0,0,0,0.00,0,0,0,0,0,10000.0,0.0,0.0,40446,10502,1250,1134,2576,1450,24.762282,0,10000.00,1,55908,45908.0,45908.0,0,0,0,1,0,30998.0,38,30998.0,06033,41.70,-72.60,0.0,0.0,0,3.442,0,10000.0,3,3,0,0,10.0
1,2743440,H,FS19,U.S. Citizen,White,M,US,MI,WAYN,48212,NaN,Electrical Engineering,2019-05-15,2019-06-25,2019-06-23,E,NaN,CMPL,2019-06-25,Frontier International Academy,231841,H,48212,MI,HAMTRAMCK,US,Engineering,313,scarber,ZONE 10B,NaN,Brandon,Scarber,HIGH,2019-06-28,2019-07-02,IN STATE,Wayne,US,Hamtramck,MI,United States,I,EFC < PELL CUTOFF,2.710,0,0,0,0,0,0,0,0,0,0.0,1,0,1,1,2020,0,400,44013,0,44013,5845,0.00,3500,2000,5500,0,0,8300.0,0.0,0.0,15708,10502,464,1134,2576,0,53.048703,0,19645.00,1,30384,22084.0,22084.0,0,0,0,1,1,100637.0,16,100637.0,48212,42.38,-83.10,0.0,0.0,0,2.168,0,8300.0,41,41,39,1,1.0
2,2589512,H,FS19,U.S. Citizen,White,M,NaN,NaN,NaN,NaN,SATX,Chemical Engineering,2019-05-13,2019-06-18,NaT,N,D,CMPL,2019-06-18,Indian Hill High School,361020,H,45243,OH,CINCINNATI,US,Engineering,R2,irissvb,Ohio/Indiana/Kentucky,Cincinnati Area,Iris,Shen-Van Buren,MODERATE,NaT,2019-07-02,OUT OF STATE,Hamilton County,US,CINCINNATI,OH,United States,O,EFC >= 20% of COA and < 60%,4.488,0,0,0,0,0,0,700,660,0,0.0,0,1,1,0,2020,1,24311,116651,1585,115066,0,0.00,3500,2000,5500,0,0,18750.0,0.0,0.0,40446,10502,1250,1134,2576,1360,46.429279,0,24250.00,1,55908,37158.0,37158.0,0,0,0,1,0,54729.0,16,54729.0,45243,39.09,-84.51,0.0,0.0,0,3.590,0,18750.0,36,36,0,0,10.0
3,2612448,H,FS19,U.S. Citizen,Black or African American,F,NaN,NaN,NaN,NaN,SATX,Hospitality Business,2019-05-07,2019-05-20,NaT,N,D,CMPL,2019-05-20,Communication & Media Arts High School,230874,H,48227,MI,DETROIT,US,Business,313,scarber,ZONE 10B,NaN,Brandon,Scarber,HIGH,NaT,2019-07-02,IN STATE,Wayne,US,Detroit,MI,United States,I,EFC < PELL CUTOFF,3.097,0,0,0,0,0,0,510,540,0,0.0,1,0,1,0,2020,1,0,25910,0,25910,6195,550.00,3500,2000,5500,0,0,10800.0,0.0,0.0,15708,10502,464,1134,2576,1050,69.027227,0,23045.00,1,30384,19584.0,19584.0,0,0,0,1,0,43988.0,08,43988.0,48227,42.21,-83.15,0.0,0.0,0,2.319,0,10800.0,13,13,0,0,14.0
4

,SCHOOL_CEEB,school_feeder
0,3,1
1,38,1
2,187,2
3,192,1
4,196,1
...,...,...
9897,821279,1
9898,826211,1
9899,826347,1
9900,826577,1


In [35]:
#display(raw_df)
#list(raw_df.columns.values)

In [42]:
display(raw_df.head(100))

,PID,LEVL,APPLICATION_TERM,CITIZENSHIP,ETHNICITY,GENDER,COUNTRY,STATE,COUNTY,ZIP_CODE,SAT_TYPE,INTENDED_DECLARED_MAJOR,APPLICATION_DATE,ADMITTANCE_DATE,DEPOSIT_DATE,MATRICULANT,DEP_STATUS,APPLICATION_REVIEW_STATUS,APPLICATION_REVIEW_DATE,SCHOOL_NAME,SCHOOL_CEEB,SCHOOL_TYPE,SCHOOL_ZIP_CODE,SCHOOL_STATE,SCHOOL_CITY,SCHOOL_COUNTRY,COLLEGE,RECRUIT_CD,RECRUITER_MSU_ID,TERRITORY_NM,TARGET_AREA,RECRUIT_FRST_NM,RECRUIT_LST_NM,NEED_LEVEL_EFC_BAND_CATEGORY,MATRICULANT_DT,EDW_CREATE_DATE,STUDENT_SEGMENT,RECRUIT_COUNTY,RECRUIT_COUNTRY,RECRUIT_CITY,RECRUIT_STATE,RECRUIT_COUNTRY_NAME,FEE_BSD_RES_TYP_CD,NEED_LEVEL_DEFINITION,HIGH_SCHOOL_GPA,ACT_COMPOSITE,ACT_ENGLISH,ACT_MATH,ACT_READING,ACT_SCIENCE,ACT_WRITING,SAT_MATH,SAT_VERBAL,SAT_WRITING,IELTS_SCORE,FIRST_GENERATION,LEGACY_STUDENT,ADMITTANCE,DEPOSIT,AIDYR,FAFSA,EFC1,TOTAL_AGI1,STUDENT_AGI1,PARENT_AGI1,PELL_GRANT,FED_GIFT_EXLUDING_PELL_AID,FED_SUB_LOAN,FED_UNSUB_LOAN,FED_TOTAL_LOAN,PLUS_LOAN,STATE_GIFT_AID,MSU_OFR_GIFT_AID,OTHER_GIFT,OTHER_LOAN,TUITION,ROOM,TRANSPORTATION,BOOKS,OTHER,SAT_COMBINED,DISCOUNT_RATE,AP_CNT_NBR,FINANCIAL_AID_TOTAL,FINANCIAL_AID_FLAG,COA,NTR_BASED_ON_OFFERED,NTR_ACCEPTED,PLACED_INTO_CALCULUS,PLACED_INTO_COLLEGE_WRITING,PLACED_INTO_COLLEGE_MATH,ADMITS,MATRICULANTS,ENTY_ID,COLGE_CD,SCHOOL_NBR,RECRUIT_ZIP_CD,LATD_NBR,LGTD_NBR,TOEFL_SCORE,ATHLETIC_AID,TUITION_WAIVER,PRED_GPA_NBR,MATH_PLACEMENT_SCOR_NBR,MSU_ACPT_GIFT_AID,Days_To_Admit,Days_Since_Application_Review,Days_To_Deposit,Yield_Indicator,school_feeder
0,2750786,H,FS19,U.S. Citizen,Asian,F,NaN,NaN,NaN,NaN,SATX,Political Science,2019-05-17,2019-05-20,NaT,N,NaN,CMPL,2019-05-20,Glastonbury High School,70215,H,06033,CT,GLASTONBURY,US,Social Science,R10,larrya,New York Metro/DC Metro,OTHER/NON TARGET,Larry,Alterman,HIGH,NaT,2019-07-02,OUT OF STATE,Hartford County,US,GLASTONBURY,CT,United States,O,EFC < 20% of COA,4.303,0,0,0,0,0,0,770,680,0,0.0,0,0,1,0,2020,0,0,0,0,0,0,0.0,0,0,0,0,0,10000.0,0.0,0.0,40446,10502,1250,1134,2576,1450,24.762282,0,10000.0,1,55908,45908.0,45908.0,0,0,0,1,0,30998.0,38,30998.0,06033,41.70,-72.60,0.0,0.0,0,3.442,0,10000.0,3,3,0,0,10.0
1,2743440,H,FS19,U.S. Citizen,White,M,US,MI,WAYN,48212,NaN,Electrical Engineering,2019-05-15,2019-06-25,2019-06-23,E,NaN,CMPL,2019-06-25,Frontier International Academy,231841,H,48212,MI,HAMTRAMCK,US,Engineering,313,scarber,ZONE 10B,NaN,Brandon,Scarber,HIGH,2019-06-28,2019-07-02,IN STATE,Wayne,US,Hamtramck,MI,United States,I,EFC < PELL CUTOFF,2.710,0,0,0,0,0,0,0,0,0,0.0,1,0,1,1,2020,0,400,44013,0,44013,5845,0.0,3500,2000,5500,0,0,8300.0,0.0,0.0,15708,10502,464,1134,2576,0,53.048703,0,19645.0,1,30384,22084.0,22084.0,0,0,0,1,1,100637.0,16,100637.0,48212,42.38,-83.10,0.0,0.0,0,2.168,0,8300.0,41,41,39,1,1.0
2,2589512,H,FS19,U.S. Citizen,White,M,NaN,NaN,NaN,NaN,SATX,Chemical Engineering,2019-05-13,2019-06-18,NaT,N,D,CMPL,2019-06-18,Indian Hill High School,361020,H,45243,OH,CINCINNATI,US,Engineering,R2,irissvb,Ohio/Indiana/Kentucky,Cincinnati Area,Iris,Shen-Van Buren,MODERATE,NaT,2019-07-02,OUT OF STATE,Hamilton County,US,CINCINNATI,OH,United States,O,EFC >= 20% of COA and < 60%,4.488,0,0,0,0,0,0,700,660,0,0.0,0,1,1,0,2020,1,24311,116651,1585,115066,0,0.0,3500,2000,5500,0,0,18750.0,0.0,0.0,40446,10502,1250,1134,2576,1360,46.429279,0,24250.0,1,55908,37158.0,37158.0,0,0,0,1,0,54729.0,16,54729.0,45243,39.09,-84.51,0.0,0.0,0,3.590,0,18750.0,36,36,0,0,10.0
3,2612448,H,FS19,U.S. Citizen,Black or African American,F,NaN,NaN,NaN,NaN,SATX,Hospitality Business,2019-05-07,2019-05-20,NaT,N,D,CMPL,2019-05-20,Communication & Media Arts High School,230874,H,48227,MI,DETROIT,US,Business,313,scarber,ZONE 10B,NaN,Brandon,Scarber,HIGH,NaT,2019-07-02,IN STATE,Wayne,US,Detroit,MI,United States,I,EFC < PELL CUTOFF,3.097,0,0,0,0,0,0,510,540,0,0.0,1,0,1,0,2020,1,0,25910,0,25910,6195,550.0,3500,2000,5500,0,0,10800.0,0.0,0.0,15708,10502,464,1134,2576,1050,69.027227,0,23045.0,1,30384,19584.0,19584.0,0,0,0,1,0,43988.0,08,43988.0,48227,42.21,-83.15,0.0,0.0,0,2.319,0,10800.0,13,13,0,0,14.0
4,2739797

In [41]:
raw_df.describe()

,HIGH_SCHOOL_GPA,ACT_COMPOSITE,ACT_ENGLISH,ACT_MATH,ACT_READING,ACT_SCIENCE,ACT_WRITING,SAT_MATH,SAT_VERBAL,SAT_WRITING,IELTS_SCORE,FIRST_GENERATION,LEGACY_STUDENT,ADMITTANCE,DEPOSIT,FAFSA,EFC1,TOTAL_AGI1,STUDENT_AGI1,PARENT_AGI1,PELL_GRANT,FED_GIFT_EXLUDING_PELL_AID,FED_SUB_LOAN,FED_UNSUB_LOAN,FED_TOTAL_LOAN,PLUS_LOAN,STATE_GIFT_AID,MSU_OFR_GIFT_AID,OTHER_GIFT,OTHER_LOAN,TUITION,ROOM,TRANSPORTATION,BOOKS,OTHER,SAT_COMBINED,DISCOUNT_RATE,AP_CNT_NBR,FINANCIAL_AID_TOTAL,FINANCIAL_AID_FLAG,COA,NTR_BASED_ON_OFFERED,NTR_ACCEPTED,PLACED_INTO_CALCULUS,PLACED_INTO_COLLEGE_WRITING,PLACED_INTO_COLLEGE_MATH,ADMITS,MATRICULANTS,LATD_NBR,LGTD_NBR,TOEFL_SCORE,ATHLETIC_AID,TUITION_WAIVER,PRED_GPA_NBR,MATH_PLACEMENT_SCOR_NBR,MSU_ACPT_GIFT_AID,Days_To_Admit,Days_Since_Application_Review,Days_To_Deposit,Yield_Indicator,school_feeder
count,56921.000000,56921.000000,56921.000000,56921.000000,56921.000000,56921.000000,56921.000000,56921.000000,56921.000000,56921.000000,56921.000000,56921.000000,56921.000000,56921.0,56921.000000,56921.000000,56921.000000,5.692100e+04,5.692100e+04,5.692100e+04,56921.000000,56921.000000,56921.000000,56921.000000,56921.000000,56921.000000,56921.000000,56921.000000,56921.000000,56921.000000,56921.000000,56921.000000,56921.000000,56921.000000,56921.000000,56921.000000,56921.000000,56921.000000,56921.000000,56921.000000,56921.000000,56921.000000,56921.000000,56921.000000,56921.00000,56921.000000,56921.0,56921.000000,56921.000000,56921.000000,56921.000000,5.692100e+04,5.692100e+04,56921.000000,56921.000000,56921.000000,56921.000000,56921.000000,56921.000000,56921.000000,56760.000000
mean,3.181290,8.666977,8.736371,8.573163,8.702412,8.479489,2.544281,443.311432,428.803078,0.686214,0.093243,0.214262,0.176051,1.0,0.303737,0.452627,20081.770015,9.820215e+04,1.266255e+03,9.693589e+04,850.015882,45.566598,929.044096,1512.430070,2441.474166,452.283867,99.490188,5465.594673,145.376912,211.884538,28113.654539,10771.699900,850.055305,1122.058643,2549.348571,872.114510,19.779108,0.531473,9711.686823,0.705926,43406.816957,37942.066585,37942.066585,0.078284,0.25813,0.171659,1.0,0.313593,34.075890,-70.555476,2.693321,3.956986e+03,2.478601e+03,2.685192,3.192460,5465.594673,57.432248,64.387133,44.660002,0.303737,84.572974
std,1.453066,12.749296,12.978490,12.664937,12.980758,12.518405,3.849977,288.001978,276.035524,18.989121,0.989663,0.410313,0.380867,0.0,0.459874,0.497755,57929.165383,1.949298e+05,2.098318e+04,1.915784e+05,1997.214387,539.616103,1560.685721,2171.047439,2873.779746,3485.885412,1020.609185,7049.350775,1392.336392,2191.443944,12376.271368,511.987167,390.139224,22.826998,51.563483,559.541457,27.594598,1.519687,10085.658192,0.455629,12949.964530,12666.083822,12666.083822,0.268620,0.43761,0.377087,0.0,0.463957,16.067012,33.926998,8.131740,1.575828e+05,5.728367e+04,1.253569,6.611016,7049.350775,66.741654,78.609039,71.383933,0.459874,115.746204
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,-9.999999e+06,-8.088410e+05,-9.999999e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-33671.500000,-33671.500000,0.000000,0.00000,0.000000,1.0,0.000000,0.000000,-159.350000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,3.301000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,15708.000000,10322.000000,464.000000,1108.000000,2518.000000,0.000000,0.000000,0.000000,0.000000,0.000000,30384.000000,30110.000000,30110.000000,0.000000,0.00000,0.000000,1.0,0.000000,37.830000,-85.650000,0.000000,0.000000e+00,0.000000e+00,2.6820

In [45]:
# Remove non-accepted students ------------- Where My issue lies 
raw_df = raw_df[raw_df.ADMITTANCE == 1]  # only keep admitted applicants
raw_df = raw_df.drop(['ADMITTANCE'], axis=1)  # remove admittance column
display(raw_df) ####

,PID,LEVL,APPLICATION_TERM,CITIZENSHIP,ETHNICITY,GENDER,COUNTRY,STATE,COUNTY,ZIP_CODE,SAT_TYPE,INTENDED_DECLARED_MAJOR,APPLICATION_DATE,ADMITTANCE_DATE,DEPOSIT_DATE,MATRICULANT,DEP_STATUS,APPLICATION_REVIEW_STATUS,APPLICATION_REVIEW_DATE,SCHOOL_NAME,SCHOOL_CEEB,SCHOOL_TYPE,SCHOOL_ZIP_CODE,SCHOOL_STATE,SCHOOL_CITY,SCHOOL_COUNTRY,COLLEGE,RECRUIT_CD,RECRUITER_MSU_ID,TERRITORY_NM,TARGET_AREA,RECRUIT_FRST_NM,RECRUIT_LST_NM,NEED_LEVEL_EFC_BAND_CATEGORY,MATRICULANT_DT,EDW_CREATE_DATE,STUDENT_SEGMENT,RECRUIT_COUNTY,RECRUIT_COUNTRY,RECRUIT_CITY,RECRUIT_STATE,RECRUIT_COUNTRY_NAME,FEE_BSD_RES_TYP_CD,NEED_LEVEL_DEFINITION,HIGH_SCHOOL_GPA,ACT_COMPOSITE,ACT_ENGLISH,ACT_MATH,ACT_READING,ACT_SCIENCE,ACT_WRITING,SAT_MATH,SAT_VERBAL,SAT_WRITING,IELTS_SCORE,FIRST_GENERATION,LEGACY_STUDENT,DEPOSIT,AIDYR,FAFSA,EFC1,TOTAL_AGI1,STUDENT_AGI1,PARENT_AGI1,PELL_GRANT,FED_GIFT_EXLUDING_PELL_AID,FED_SUB_LOAN,FED_UNSUB_LOAN,FED_TOTAL_LOAN,PLUS_LOAN,STATE_GIFT_AID,MSU_OFR_GIFT_AID,OTHER_GIFT,OTHER_LOAN,TUITION,ROOM,TRANSPORTATION,BOOKS,OTHER,SAT_COMBINED,DISCOUNT_RATE,AP_CNT_NBR,FINANCIAL_AID_TOTAL,FINANCIAL_AID_FLAG,COA,NTR_BASED_ON_OFFERED,NTR_ACCEPTED,PLACED_INTO_CALCULUS,PLACED_INTO_COLLEGE_WRITING,PLACED_INTO_COLLEGE_MATH,ADMITS,MATRICULANTS,ENTY_ID,COLGE_CD,SCHOOL_NBR,RECRUIT_ZIP_CD,LATD_NBR,LGTD_NBR,TOEFL_SCORE,ATHLETIC_AID,TUITION_WAIVER,PRED_GPA_NBR,MATH_PLACEMENT_SCOR_NBR,MSU_ACPT_GIFT_AID,Days_To_Admit,Days_Since_Application_Review,Days_To_Deposit,Yield_Indicator,school_feeder
0,2750786,H,FS19,U.S. Citizen,Asian,F,NaN,NaN,NaN,NaN,SATX,Political Science,2019-05-17,2019-05-20,NaT,N,NaN,CMPL,2019-05-20,Glastonbury High School,70215,H,06033,CT,GLASTONBURY,US,Social Science,R10,larrya,New York Metro/DC Metro,OTHER/NON TARGET,Larry,Alterman,HIGH,NaT,2019-07-02,OUT OF STATE,Hartford County,US,GLASTONBURY,CT,United States,O,EFC < 20% of COA,4.303,0,0,0,0,0,0,770,680,0,0.0,0,0,0,2020,0,0,0,0,0,0,0.00,0,0,0,0,0,10000.0,0.0,0.0,40446,10502,1250,1134,2576,1450,24.762282,0,10000.00,1,55908,45908.0,45908.0,0,0,0,1,0,30998.0,38,30998.0,06033,41.70,-72.60,0.0,0.0,0,3.442,0,10000.0,3,3,0,0,10.0
1,2743440,H,FS19,U.S. Citizen,White,M,US,MI,WAYN,48212,NaN,Electrical Engineering,2019-05-15,2019-06-25,2019-06-23,E,NaN,CMPL,2019-06-25,Frontier International Academy,231841,H,48212,MI,HAMTRAMCK,US,Engineering,313,scarber,ZONE 10B,NaN,Brandon,Scarber,HIGH,2019-06-28,2019-07-02,IN STATE,Wayne,US,Hamtramck,MI,United States,I,EFC < PELL CUTOFF,2.710,0,0,0,0,0,0,0,0,0,0.0,1,0,1,2020,0,400,44013,0,44013,5845,0.00,3500,2000,5500,0,0,8300.0,0.0,0.0,15708,10502,464,1134,2576,0,53.048703,0,19645.00,1,30384,22084.0,22084.0,0,0,0,1,1,100637.0,16,100637.0,48212,42.38,-83.10,0.0,0.0,0,2.168,0,8300.0,41,41,39,1,1.0
2,2589512,H,FS19,U.S. Citizen,White,M,NaN,NaN,NaN,NaN,SATX,Chemical Engineering,2019-05-13,2019-06-18,NaT,N,D,CMPL,2019-06-18,Indian Hill High School,361020,H,45243,OH,CINCINNATI,US,Engineering,R2,irissvb,Ohio/Indiana/Kentucky,Cincinnati Area,Iris,Shen-Van Buren,MODERATE,NaT,2019-07-02,OUT OF STATE,Hamilton County,US,CINCINNATI,OH,United States,O,EFC >= 20% of COA and < 60%,4.488,0,0,0,0,0,0,700,660,0,0.0,0,1,0,2020,1,24311,116651,1585,115066,0,0.00,3500,2000,5500,0,0,18750.0,0.0,0.0,40446,10502,1250,1134,2576,1360,46.429279,0,24250.00,1,55908,37158.0,37158.0,0,0,0,1,0,54729.0,16,54729.0,45243,39.09,-84.51,0.0,0.0,0,3.590,0,18750.0,36,36,0,0,10.0
3,2612448,H,FS19,U.S. Citizen,Black or African American,F,NaN,NaN,NaN,NaN,SATX,Hospitality Business,2019-05-07,2019-05-20,NaT,N,D,CMPL,2019-05-20,Communication & Media Arts High School,230874,H,48227,MI,DETROIT,US,Business,313,scarber,ZONE 10B,NaN,Brandon,Scarber,HIGH,NaT,2019-07-02,IN STATE,Wayne,US,Detroit,MI,United States,I,EFC < PELL CUTOFF,3.097,0,0,0,0,0,0,510,540,0,0.0,1,0,0,2020,1,0,25910,0,25910,6195,550.00,3500,2000,5500,0,0,10800.0,0.0,0.0,15708,10502,464,1134,2576,1050,69.027227,0,23045.00,1,30384,19584.0,19584.0,0,0,0,1,0,43988.0,08,43988.0,48227,42.21,-83.15,0.0,0.0,0,2.319,0,10800.0,13,13,0,0,14.0
4,2739797,H,FS19,Non